# REKT Bet
#### The Best Crypto-Gambling App 

In [90]:
import pandas as pd
import numpy as np
%matplotlib inline
from datetime import date
from datetime import datetime
from datetime import timedelta 
from dateutil.relativedelta import relativedelta
import re 
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder

import os
import pprint
import binascii
import mnemonic
import bip32utils
from py_crypto_hd_wallet import HdWallet, HdWalletSaver, HdWalletFactory, HdWalletCoins, HdWalletSpecs, HdWalletWordsNum
from bipwallet import wallet
from py_hd_wallet import wallet
import ast
import subprocess
import json
from pathlib import Path
import subprocess
import requests
from dotenv import load_dotenv

from constants import *
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from web3 import Web3, HTTPProvider
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy
from eth_account import Account
load_dotenv()



True

# The NBA API

In [91]:
# pulling data from nba_api
games = leaguegamefinder.LeagueGameFinder()

In [92]:
start_date = date.today().strftime('%Y-%m-%d')
end_date = (date.today() + relativedelta(months=+3)).strftime('%Y-%m-%d')

# Converting to pandas df
find_games = games.get_data_frames()[0]
All_games = pd.DataFrame(find_games)
All_games['GAME_DATE'] = pd.to_datetime(All_games['GAME_DATE']) 
sorted_games = All_games.set_index('GAME_DATE')
#game_dates = datetime.strptime(sorted_games.index, '%Y-%m-%d').date()
game_table = sorted_games[:(date.today() + relativedelta(days=-2)).strftime('%Y-%m-%d')]
game_table

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,MATCHUP,WL,MIN,PTS,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
GAME_DATE,,,,,,,,,,,,,,,,,,,,,
2020-09-03,22020,1611661313,NYL,New York Liberty,1022001106,NYL vs. ATL,None,159,49,18,...,0.583,11,30,41,9,6,5,10,9,-3.8
2020-09-03,42019,1610612738,BOS,Boston Celtics,0041900213,BOS vs. TOR,None,120,57,21,...,0.750,5,24,29,10,2,6,7,9,10.2
2020-09-03,22020,1611661330,ATL,Atlanta Dream,1022001106,ATL @ NYL,None,161,54,23,...,1.000,9,34,43,11,4,8,9,11,3.4
2020-09-03,42019,1610612761,TOR,Toronto Raptors,0041900213,TOR @ BOS,None,119,47,18,...,0.545,3,18,21,11,4,3,7,9,-10.4
2020-09-02,22020,1611661322,WAS,Washington Mystics,1022001105,WAS @ SEA,L,200,64,26,...,1.000,5,20,25,19,5,3,7,16,-7.0
2020-09-02,22020,1611661328,SEA,Seattle Storm,1022001105,SEA vs. WAS,W,202,71,24,...,0.818,11,32,43,18,5,2,13,10,7.0
2020-09-02,42019,1610612745,HOU,Houston Rockets,0041900177,HOU vs. OKC,W,240,104,35,...,0.895,11,33,44,18,11,7,12,24,2.0
2020-09-02,42019,1610612760,OKC,Oklahoma City Thunder,0041900177,OKC @ HOU,L,241,102,35,...,0.762,7,39,46,24,6,4,21,16,-2.0
2020-09-02,22020,1611661329,CHI,Chicago Sky,1022001103,CHI vs. MIN,L,200,83,33,...,0.889,8,29,37,23,11,6,23,16,-3.0


#### So User will choose a team. The team chosen is then sorted among the df['TEAM_NAME']. When API updates after the game, the rows of the team chosen will update their df['WL']. From none to L or W. That information cell will then be sent to the contract and placed in the getTeamVictor() function, which will allow the contract to distribute funds to the winners.

#### make function that will send a value to contract at a specific time. ie when game starts so bets can close and when game ends to release funds.

In [ ]:
game_table['MATCHUP']= game_table['MATCHUP'].str.replace(r'(\w)\1{%d,}'%(min_threshold_rep-1), r'\1')


In [53]:
if game_table[start_date] 

# Splitting characters in String  
res = re.split(', |@|-|vs.', <df-name>)

for game_table[start_date]

'2020-08-31'

# The Blockchain

In [93]:
# create web3 interface

w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))           # uses gnache and remix on local machine

#w3.middleware_onion.inject(geth_poa_middleware, layer=0)
#w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

#w3 = Web3(Web3.HTTPProvider('https://ropsten.infura.io/v3/1801fc318b4741a687227c1e9d594c49'))

In [94]:
w3.eth.accounts[0]

'0x1a708Ac9BBFFd62fdd668F8747CA2611415476BC'

In [66]:
# testing with account 0
#w3.eth.accounts[0]
w3.isConnected()

True

In [67]:
w3.eth.blockNumber

97

# Working Code Starts Here

In [75]:
# compiles contract using cmd line
def compile_contract():
    command = f'solc REKT-Gamble-.sol' 
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
   
    return output

In [83]:
# Outputs contract bytecode like in Remix.  --combined-json other outputs: asm,ast,bin,bin-runtime,compact-format,devdoc,hashes,interface,metadata,opcodes,srcmap,srcmap-runtime,storage-layout,userdoc
def contract_bytecode():
    command = f'solc --combined-json bin,opcodes,srcmap REKT-Gamble-.sol' 
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    
    return json.loads(output)

# Outputs contract bin
def contract_bin():
    command = f'solc --combined-json bin REKT-Gamble-.sol' 
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    j1_load = json.loads(output)
    contract_bin = j1_load['contracts']['REKT-Gamble-.sol:REKT-Gamble-']['bin']
    return contract_bin

# Outputs contract abi
def contract_abi():
    command = f'solc --combined-json abi,bin REKT-Gamble-.sol'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    j2_load = json.loads(output)
    contract_abi = j2_load['contracts']['REKT-Gamble-.sol:REKT-Gamble-']['abi']
    return json.loads(contract_abi)

In [86]:
command = f'solc --combined-json bin,opcodes,srcmap REKT-Gamble-.sol' 
p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
(output, err) = p.communicate()
output

b''

In [88]:
# Deploy contract                                                # the help function that i always forget about
                                                                # help(TieredProfitSplitter.constructor)
def deploy_contract():
    global contract_address, contract
    
    #  default account as the sender
    w3.eth.defaultAccount = w3.eth.accounts[0]
    contract = w3.eth.contract(abi=contract_abi(),
                              bytecode=contract_bin())
    
    # Instantiating the smart contract
    tx_hash = contract.constructor().transact()
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    #stores contract address and makes it global
    
    contract_address = tx_receipt.contractAddress
    
    return tx_receipt

In [89]:
def distributePrizesPY(winner):
    tx_hash_dp = contract.functions.distributePrizes(f'{winner}').transact()
    tx_receiptdp = w3.eth.waitForTransactionReceipt(tx_hash)
    return tx_receiptdp

# Interaction Between Contract and Py Starts here

In [72]:
# call functions means we're reading data from the blockchain
# send functions means we're going be writing data

In [73]:
# Turning Solidity function into a python function
def deposit_py(amount):
    if amount.isdigit() == True:
        # remix and python sometimes dont match in lower/upper case addresses so we must use checkSum
        address_cs = w3.toChecksumAddress(contract_address)
        w3.eth.defaultAccount = w3.eth.accounts[1]

        #initializing the contract
        global tiered_contract
        tiered_contract = w3.eth.contract(address=address_cs, abi=contract_abi())

        # making a transaction by calling the deposit() function and then using transact with a provided amount to make transaction.
        depo_hash = tiered_contract.functions.deposit().transact({'from':w3.eth.defaultAccount,'value': w3.toWei({amount},'ether')})
        depo_receipt = w3.eth.waitForTransactionReceipt(depo_hash)
        return deposit_receipt
    else:
        return print("Amount is invalid!")

def balance_py():
    # calling balance function from sol contract. Should always return in 0
    Balance = tiered_contract.functions.balance().call()
    return Balance

In [74]:
deposit_py(9)

AttributeError: 'int' object has no attribute 'isdigit'

In [ ]:
# This will create the transaction raw and unsigned. Serves as a preview transaction before sending it.
def create_tx(coin, account, recipient, amount):
    global tx_data
    if coin =='eth':
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        tx_data = {
            "from": account.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
        }
        return tx_data
    elif coin == 'btc':
        tx_data = PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])
        return tx_data
    else:
        return print("Please use lower case. If this error persists, please use another coin.")
    
# Signs the transaction created and sends it to a designated network
def send_tx(coin, account, recipient, amount):
    
    if coin == 'eth':
        tx = create_tx(coin,account, recipient, amount)
        signed_tx = account.sign_transaction(tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == 'btc':
        #tx_send = create_tx(coin, account, recipient, amount)
        signed = account.sign_transaction(tx_data)
        NetworkAPI.broadcast_tx_testnet(signed)
        return signed